In [70]:
import pandas as pd
import glob
import os
from functools import reduce

Due to the fact that the "DATE" column's format in different feature csv data being different from different online sources, the above merge would not be able to happen properly, resulting the dataframe to only have the merged column names but no tuple data rows.

In [71]:
# Thus, the following is a new way of joining our data into one single dataframe object.
# Find all names of our data files in csv format (all csv data has the format of 2 columns, 1st column being DATE, 
# Second column being the feature we believe that is important for our model). In order to merge all these seperate
# featrue values into a single csv (pandas DataFrame) file for futher data cleaning and processing before the modeling,
# phase, we loop through all the file names (saved into a list) and reading them into the pandas DataFrame one at a 
# time while joining them based on the "DATE" column.

# Initialize an empty DataFrame object to build up into a complete dataframe with all needed features through merging other feature dataframes
df = pd.DataFrame()
# locate the directory path where the data are located on the local machine
wd = os.path.abspath('FeatureData')
# find all the data in csv format under the located directory and save them as a list variable
all_files = glob.glob(wd + '/*.csv')
# Open all the csv feature data as a Pandas DataFrame and saving it inside a list variable
df_list = [pd.read_csv(file) for file in all_files]
# Expand the initialized DataFrame by assigning it the second column of all the feature data file as a new column
# omitting the DATE column to avoid problems caused by different DATE format
for fileIndex in range(len(df_list)):
    df[df_list[fileIndex].columns[1]] = df_list[fileIndex][df_list[fileIndex].columns[1]]
# Add the DATE column into the expanded dataframe with the correct format (the same as our training and testing dataset)
##### Continued on the next few cells #####

df.head(5)



,ASPNHSUS,UNRATE,USACPIHOUMINMEI,FEDFUNDS,CSUSHPINSA,MSPNHSUS,MSACSR,MORTGAGE30US,NHFSEPUC,NHSDPC,NHSDPNS,HNFSEPUSSA,HSN1F,DSPIC96,TOTALSA
0,144200.0,7.3,59.963759,4.03,75.697,120000.0,5.2,8.4320,131.0,16.0,17.0,281.0,676.0,6616.3,12.6
1,144800.0,7.4,60.195578,4.06,75.652,117200.0,4.9,8.7625,129.0,16.0,20.0,269.0,639.0,6649.9,12.9
2,144800.0,7.4,60.466033,3.98,75.812,120000.0,6.1,8.9350,135.0,16.0,20.0,279.0,553.0,6659.6,12.8
3,145000.0,7.4,60.388760,3.73,76.079,120000.0,6.1,8.8525,134.0,14.0,17.0,274.0,546.0,6679.4,12.6
4,146000.0,7.6,60.350123,3.82,76.398,113000.0,6.0,8.6720,135.0,16.0,18.0,273.0,554.0,6712.9,13.1


In [72]:
# The format of DATES in the training and testing data sets provided by synchrony FINANCIAL are in the for of:
#                                                  '01/month/year'
# As all other feature data are found with a corresponding DATE with Month being from January (01) ~ December (12)
# and Year from 1992 (92) ~ 2017 (17) and with the first day of the month (01).
# Thus, the next step is to create a pandas Series object representing the DATE in the corresponding string format

# create the month list with strings representing 1 ~ 12 with a '0' prefix
month = ['0%d' % s for s in range(1,13) ]
# create a new list with the substring constructed by the last two characters in the string to fit the month string 
# representation in the testin and training data sets and we are done for the month
months = [m[-2:] for m in month]
months

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

In [73]:
# As above with the month, we do the similar process with the year.
# creat a list of string representation of the year with range from 1992 ~ 2017
year = [str(y) for y in range(1992, 2018)]
# create the years list with the correct format of the year's strings by saving all the year element's substring 
# constructed by the last two characters of the string
years = [y[-2:] for y in year]

In [74]:
# Create the DATE list with the same string formats as the training and testing data set
# by looping through the months and years list 
DATE = ['01/%s/%s' % (month, year) for year in years for month in months]
# Turn the list into a Pandas Series Object so we can join it with the main dataframe
DATE = pd.Series(DATE)
# Join the DATE Series with the main dataframe, so the dataframe would have the "DATE" column
df["DATE"] = DATE

In [75]:
df.tail()

,ASPNHSUS,UNRATE,USACPIHOUMINMEI,FEDFUNDS,CSUSHPINSA,MSPNHSUS,MSACSR,MORTGAGE30US,NHFSEPUC,NHSDPC,NHSDPNS,HNFSEPUSSA,HSN1F,DSPIC96,TOTALSA,DATE
307,369200.0,4.4,120.448724,1.16,194.684,314200.0,6.0,3.880,176.0,16.0,13.0,280.0,559.0,12785.4,16.4,01/08/17
308,379300.0,4.2,120.728451,1.15,195.155,331500.0,5.3,3.805,175.0,17.0,14.0,280.0,639.0,12786.9,18.9,01/09/17
309,394000.0,4.1,121.139930,1.15,195.430,319500.0,5.6,3.895,176.0,18.0,13.0,286.0,616.0,12805.3,18.4,01/10/17
310,388500.0,4.1,121.279407,1.16,195.815,343400.0,4.8,3.922,181.0,17.0,16.0,287.0,711.0,12814.8,17.9,01/11/17
311,398700.0,4.1,121.604340,1.30,196.216,340100.0,5.4,3.950,176.0,19.0,13.0,295.0,653.0,12846.3,18.2,01/12/17


Working with feature data collected in a seasonal matter

In [76]:
# We have a few features which are recorded in a seasonal style, thus we will expand them so that the data for each
# sesaon would be the same for each months in that season.

In [77]:
# In order to work with the Seasonal Data, need to change it into monthly presentation (Each tuple data times three)
# We solve it by replicating each row of data 3 times and saving these replicated data into a new list
# then transform the list into a pandas Series. This would expand the seasonal data into monthly data.
# then we simply join this series into the main dataframe as a new column

# change into the directory where the seasonal Data featrues csv data files are located
wd = os.path.abspath("Seasonal Data to be changed")
# save all the csv files in the files_list
files_list = glob.glob( wd + '/*.csv')
# read all files into an dataframe and save dataframes to the df_list
df_list = [pd.read_csv(file) for file in files_list]
# create the data_list where we will put the monthly data expanded by the seasonal data in
data_list = []
# loop through each of the pandas dataframe to access the feature data Series (all based on the column with index 1)
for i in range(len(df_list)):
    seasonal_data_series = df_list[i][df_list[i].columns[1]]
    # for the data values in these data seriesexpand each of them (appending) 3 times into the data_list
    for data in seasonal_data_series:
        for expand in range(3):
            data_list.append(data)
    # create a pandas Series from the data_list
    new_data_series = pd.Series(data_list)
    # Add the new pandas series into the main dataframe as a new column
    df[df_list[i].columns[1]] = new_data_series


In [78]:
# save the dataframe from memory to local disk as "AllFeaturesData.csv"
df.to_csv("AllFeaturesData.csv", index = False)

In [79]:
# Adding in the target variable to the main data file.
# To do so we append the 2 feature files of training and testing dataset, appending testing data set after the training
# data set so that the DATE order aligns with the main merged feature data file.

# read in the csv file we which we used OpenRefine to change all string representation of numbers to numeric data
df = pd.read_csv("AllFeaturesData-csv.csv")

In [80]:
# change into the directory where the Testing and Training data are located
wd = os.path.abspath("TrainTestingData")
# save all the excel (.xlsm) files in the files_list
files_list = glob.glob( wd + '/*.xlsm')
# Individually pick out the training and testing dataset
files_list[1] # Training dataset with DATES from (1992/01/01 ~ 2015/12/01)
files_list[0] # Testing Dataset with DATES from (2016/01/01 ~ 2017/12/01)
# read them into dataframes
df_train = pd.read_excel(files_list[1])
df_test = pd.read_excel(files_list[0])
# Instantiate an empty dataframe used to append both training and testing data set to it to extract the target variable's
# value as pandas series object for merging into the main dataframe later
df_main = pd.DataFrame()
df_main = df_main.append(df_train)
df_main = df_main.append(df_test)
# reindex to to get rid of duplicated index so we can merge the needed feature column to the main dataframe
df_main = df_main.reset_index()
# Merge it into the main dataframe, and change its type to integer
df[df_main.columns[2]] = df_main[df_main.columns[2]]
map(int,df[df_main.columns[2]])


# Save dataframe to "AllFeaturesData-csv.csv", ommitting its index
df.to_csv("AllFeaturesData-csv.csv", index = False)

In [81]:
# deep copy the original dataframe into another dataframe named df2 for splitting it into the training and testing data
# set based on the index (training set has the DATE starting from (1992/01/01 ~ 2015/12/01)
# while the testing set has the DATE starting from (2016/01/01 ~ 2017/12/01))
df2 = df.copy()
# The dataframe for the training dataset with index 0 to the last index of 12 * (2017 - 2015) [since each year has 12
# months, meaning for the last two years, which belong in the testing set has 12 * 2Years of rows(which is also
# the numbers of index)]. Similar with the testing set with the last 12*(2017-2015) rows (number of index)
df_train = df2[:-12*(2017-2015)]
df_test = df2[-12*(2017-2015):]
# Next we have to save these two dataframes into a CSV file under the "TrainTestingData" directory where the original
# test and train data sets are (though with only the DATE and Sales in $MM features)

# Save the current working directory path in the variable current_working_directory so that after changing into the 
# "TrainTestingData" directory and saving the files we can change back into this directory to avoid bugs created by
# still being in the "TrainTestingData" directory if we rerun this program several times
current_working_directory = os.path.abspath(os.curdir)

# Find the path of the directory "TrainTestingData"
Train_Testing_Data_Path = os.path.abspath("TrainTestingData")
# change the working directory to that path
os.chdir(Train_Testing_Data_Path)
# save the dataframes to the directory
df_train.to_csv("training_set_with_features.csv", index = False)
df_test.to_csv("testing_set_with_features.csv", index = False)

# Change back into our previous directory where it all started
os.chdir(current_working_directory)